In [ ]:
import imageio
import os
import torch

import ccc
import utils

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.cm as cm
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from affine import Affine
from datetime import datetime
from matplotlib.transforms import offset_copy
from sklearn.metrics import f1_score, precision_score, matthews_corrcoef, recall_score

from utils_plots import cmap_map

In [ ]:
df_elevation = utils.getElevationMap()

In [ ]:
TARGET_MODE = 1
MODELDIR = "2022_02_21__11-11__ALDIS_paper"

In [ ]:
tmsubdir = f'targetmode_{TARGET_MODE}'
model_root_tm_path = os.path.join(ccc.MODEL_ROOT_PATH, tmsubdir)
    
modelpath = os.path.join(model_root_tm_path, MODELDIR)
    
opt_threshold = 0.8708981871604919

_, model_name = utils.load_model(os.path.join(tmsubdir, MODELDIR), torch.device("cpu"), 18)

shap_path = os.path.join(modelpath, f"{model_name}_shap_parquet")

df_path = os.path.join(modelpath, f"{model_name}_test_df.pickle")

output_path = os.path.join(modelpath, f"{model_name}_plots_casestudy_reworked")

In [ ]:
df_test = pd.read_pickle(df_path)

In [ ]:
df_test.columns

In [ ]:
#conversion to datetime
df_test["datetime"] = pd.to_datetime(df_test[['year', 'month', 'day', 'hour']])
df = df_test[["datetime", "latitude", "longitude", "target", "output", "day", "hour"]]
df.sort_values(by=['datetime'], ascending= True)
df = df.assign(category = "")

In [ ]:
#column category
vc_threshold = utils.getVeryConfidentThreshold(opt_threshold)

df['category'] = np.select(
 [
    (df['output'] >= vc_threshold) & (df['target'] > 0.5),
    (df['output'] < vc_threshold) & (df['output'] >= opt_threshold) & (df['target'] > 0.5),
    (df['output'] < opt_threshold) & (df['target'] > 0.5),
    (df['output'] >= opt_threshold) & (df['target'] < 0.5),
    (df['output'] < opt_threshold) & (df['target'] < 0.5)
 ],
 [
    'TP_VC', # very confident true positive
    'TP_LC', # less confident true positive
    'FN',    # false negative
    'FP',    # false positive
    'TN'    # true negative
 ],
 default='ERROR'
)


In [ ]:
#Neue Spalten anlegen: y_true, y_pred

df['y_true'] = df['target'].astype(np.int32)
df['y_pred'] = (df['output'] >= opt_threshold).astype(np.int32)
df

In [ ]:
longs = []
lats = []
mccs = []
f1scores = []
precisionscores = []
recallscores = []

for long in np.unique(df.longitude):
    for lat in np.unique(df.latitude):        
        test = df[(df.longitude == long) & (df.latitude == lat)]
        f1score= f1_score(test['y_true'], test['y_pred'])
        mcc = matthews_corrcoef(test['y_true'], test['y_pred'])
        precision = precision_score(test['y_true'], test['y_pred'])
        recall = recall_score(test['y_true'], test['y_pred'])
        
        f1scores.append(f1score)
        mccs.append(mcc)
        precisionscores.append(precision)
        recallscores.append(recall)
        longs.append(long)
        lats.append(lat)
        
df_scores = pd.DataFrame({"longitude" : longs, "latitude" : lats, "mcc" : mccs, "f1score" : f1scores, "recall" : recallscores, "precision" : precisionscores})


In [ ]:
os.makedirs(output_path, exist_ok=True)

palette = "cool"
score_col = "precision"

fig = plt.figure(figsize=(19.2, 10.8))
plt.axis('off')

ax = plt.axes([0, 0, 1, 1], projection=ccrs.Mercator(), frameon=False)
ax.set_title(score_col)
ax.patch.set_visible(False)

ax.set_extent([8, 17, 45, 50], crs=ccrs.Geodetic())
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.LAKES, alpha=0.5)

ax.scatter(df_scores["longitude"], df_scores["latitude"], c=df_scores[score_col], marker='.', s=df_scores[score_col] * 1500, alpha=0.7, linewidth=2, label=score_col, cmap=palette, transform=ccrs.Geodetic())

normalize = mcolors.Normalize(vmin=df_scores[score_col].min(), vmax=df_scores[score_col].max())

scalarmappaple = cm.ScalarMappable(cmap=plt.colormaps[palette], norm=normalize)
plt.colorbar(scalarmappaple, ax=ax)

fig.savefig(os.path.join(output_path, f"quality_{score_col}.png"))
                         
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

nodes = [0.0, opt_threshold - 0.05, opt_threshold, opt_threshold + 0.01, 1.0]
colors = ["darkred", "lightcoral", "grey", "darkolivegreen", "darkgreen"]
cmap1 = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))
cmap1

In [ ]:
light_pink_r = cmap_map(lambda x: 0.3 + x / 1.5, cm.pink_r)

light_pink_r

In [ ]:
os.makedirs(output_path, exist_ok=True)

legend_TP_VC = mlines.Line2D([], [], marker="D", markersize=14, alpha=1, label='TP_VC', color="darkgreen", markeredgecolor='darkgreen', linewidth=2, linestyle='None')
legend_TP_LC = mlines.Line2D([], [], marker="D", markersize=12, alpha=0.95, color="darkolivegreen", label='TP_LC', linestyle='None')
legend_FN = mlines.Line2D([], [], marker="D", markersize=12, alpha=0.7, label='FN', color="lightcoral", markeredgecolor='darkred', linewidth=2, linestyle='None')
legend_FP = mlines.Line2D([], [], marker='.', markersize=12, alpha=0.7, label='FP', color="darkolivegreen", linestyle='None')

legend_elements = [legend_TP_VC, legend_TP_LC, legend_FN, legend_FP]

for specdate in np.unique(df_test.datetime.dt.date)[19:20]:
    for hour in [18]: #range(24):
        date = datetime(specdate.year, specdate.month, specdate.day, hour)
        test = df[df.datetime == date]

        fig = plt.figure(figsize =(19.2, 10.8))
        ax.patch.set_visible(False)
        ax = plt.axes([0, 0, 1, 1], projection=ccrs.Mercator(), frameon=False)
        ax.patch.set_visible(False)
        ax.legend(handles=legend_elements, loc="upper left", fontsize=15)
               
        # Limit the extent of the map to a small longitude/latitude range.

        ax.set_extent([7.9, 17.1, 44.9, 50.1], crs=ccrs.Geodetic())
        
        ax.scatter(df_elevation["longitude"], df_elevation["latitude"], c=df_elevation["elevation"], transform=ccrs.Geodetic(), cmap=light_pink_r, alpha=0.4)
        ax.gridlines(linestyle="--", xlocs=np.arange(8 - 0.125, 17.25, 0.25), ylocs=np.arange(45 - 0.125, 50.25, 0.25), color="grey")

        TP_VC = test[test.category == "TP_VC"] 
        TP_LC = test[test.category == "TP_LC"]
        FN = test[test.category == "FN"]
        FP = test[test.category == "FP"]
        TN = test[test.category == "TN"]
        
        ax.add_feature(cfeature.LAND)
        #ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle='-')
        #ax.add_feature(cfeature.LAKES, alpha=0.5)
        
        ax.scatter(TP_VC["longitude"], TP_VC["latitude"], c=TP_VC['output'], marker="D", s=200, alpha=1, edgecolor='darkgreen', linewidth=2, label='TP_VC', transform=ccrs.Geodetic(), cmap=cmap1, vmin=0, vmax=1)
        ax.scatter(TP_LC["longitude"], TP_LC["latitude"], c=TP_LC['output'], marker="D", s=150, alpha=1, label='TP_LC', linewidth=0, transform=ccrs.Geodetic(), cmap=cmap1, vmin=0, vmax=1)
        ax.scatter(FN["longitude"], FN["latitude"], c=FN['output'], marker="D", s=150, alpha=1, edgecolor='darkred', linewidth=2, label='FN', transform=ccrs.Geodetic(), cmap=cmap1, vmin=0, vmax=1)
        ax.scatter(FP["longitude"], FP["latitude"], c=FP['output'], marker='.', s=150, alpha=1, linewidth=0, label='FP', transform=ccrs.Geodetic(), cmap=cmap1, vmin=0, vmax=1)

        # Adding text
        # Positions for the date
        geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
        text_transform = offset_copy(geodetic_transform, units='dots')
        ax.text(16.8, 49.7, f"{date.strftime('%m/%d/%Y, %H:00')}",
                verticalalignment='center', horizontalalignment='right',transform=text_transform, fontsize = 20)

        fig.savefig(os.path.join(output_path, f"frame_{specdate}_{hour:02d}.png"), dpi=100, facecolor='auto', bbox_inches='tight')
        
        ax.clear()

In [ ]:
#Animation
import os

images = []

for file_name in sorted(os.listdir(output_path)):
    if file_name.endswith('.png'):
        file_path = os.path.join(output_path, file_name)
        img = imageio.imread(file_path, pilmode="RGB")
        images.append(img)

In [ ]:
with imageio.get_writer(os.path.join(output_path, 'animation_fast.avi'), fps=24) as writer:
    for im in images:
        writer.append_data(im)